# 수정해야 할 사안
- 의대메뉴 가져오기 안됨

# import

In [1]:
import sys
import pymysql
pckg_path = "C:/Users/22668/Desktop"
sys.path.append(pckg_path)
from jbot.config.DatabaseConfig import *
from jbot.config.PathConfig import *
PathConfig()
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
from html_table_parser import parser_functions as pf
from urllib.request import urlopen
import requests
import re

raw_chatbot_train_path1 = C:/Users/22668/Desktop/jbot/train_tools/qna/교내학식.xlsx
raw_chatbot_train_path2 =  C:/Users/22668/Desktop/jbot/train_tools/qna/교내공지.xlsx
raw_chatbot_train_path3 = C:/Users/22668/Desktop/jbot/train_tools/qna/위치.csv
raw_chatbot_train_path4 = C:/Users/22668/Desktop/jbot/train_tools/qna/sns.xlsx
raw_chatbot_train_path5 = C:/Users/22668/Desktop/jbot/train_tools/qna/과사무실.csv
raw_chatbot_train_path6 = C:/Users/22668/Desktop/jbot/train_tools/qna/교내식당 운영시간.xlsx
train_file = C:/Users/22668/Desktop/jbot/train_tools/qna/train_data.xlsx // 챗봇학습용(답변용)데이터
user_dic_path = C:/Users/22668/Desktop/jbot/utils/jbot_dict.tsv // 사용자정의사전
corpus_path = C:/Users/22668/Desktop/jbot/train_tools/dict/corpus.xlsx // 의도분류모델(딥러닝모델)학습용데이터
chatbot_dict_path = C:/Users/22668/Desktop/jbot/train_tools/dict/chatbot_dict.bin // 단어-정수 매핑데이터(pytorch의 lookuptable 또는 word2index)
intent_model_path = C:/Users/22668/Desktop/jbot/models/intent/intent_model.h5 // 학습된 의도분류모델
intent_mapping_path = C:/Users/2266

# 챗봇학습용(답변) 데이터 만들기
- 클라이언트 질문입력 -> 챗봇서버 의도,개체,시점,세부위치 분석 -> 답변,답변이미지를 DB에서 찾은 후 출력

In [2]:
#챗봇 답변(학습용) 데이터 만들기
#딥러닝의 학습용 데이터와 전혀 다른 개념!

#각각의 사람에게 받은 rawdata 불러오기
train1 = pd.read_excel(raw_chatbot_train_path1)
train2 = pd.read_excel(raw_chatbot_train_path2)
train3 = pd.read_csv(raw_chatbot_train_path3)
train4 = pd.read_excel(raw_chatbot_train_path4)
train5 = pd.read_csv(raw_chatbot_train_path5)
train6 = pd.read_excel(raw_chatbot_train_path6)

## 식당 운영시간 답변 수정

In [3]:
answers = []
for _,obs in train6.iterrows():
    answer = ""
    txt_head = obs["개체"]
    txt_last = obs["답변"]
    answer += txt_head + "\n"
    answer += txt_last
    answers.append(answer)
train6["답변"] = answers


## 과사무실 답변 수정

In [4]:
answers = []

for _,obs in train5.iterrows():
    answer = ""
    txt_head = obs["개체"] + " 사무실"
    txt_last = obs["답변"]
    answer += txt_head + "\n"
    answer += txt_last
    answers.append(answer)
train5["답변"] = answers

In [5]:
print(train5["답변"][30])

IT응용시스템공학과 사무실
사무실 : 공대 1호관 142호
T 063-270-4773, 4778
F 063-270-4542


## sns안내 전체데이터 만들기

In [6]:
def create_snstotal():
    train4 = pd.read_excel(raw_chatbot_train_path4)
    colnames = train4.columns
    sns_ner = train4["개체"].unique()
    answers = train4["답변"].unique()
    #print(train4)
    answer = ""
    for idx in range(len(sns_ner)):
        answer += sns_ner[idx] + "instagram" + "\n"
        answer += answers[idx] +"\n\n"
    _df = pd.DataFrame(data = [["SNS안내","전체",pd.NA,pd.NA,pd.NA,pd.NA,answer,pd.NA]],columns = colnames)
    train4 = pd.concat((train4,_df),axis=0).reset_index(drop=True)
    
    #print(train4)
    return train4
train4 = create_snstotal()
train4

,의도,개체,시점,요일,세부위치,질문,답변,답변이미지
0,SNS안내,총학생회,NaN,NaN,NaN,총학생회인스타보여줘,https://www.instagram.com/jbnu_ch/,NaN
1,SNS안내,총학생회,NaN,NaN,NaN,총학생회 sns알려줘,https://www.instagram.com/jbnu_ch/,NaN
2,SNS안내,총학생회,NaN,NaN,NaN,총학 인스타 주소알려줘,https://www.instagram.com/jbnu_ch/,NaN
3,SNS안내,총학생회,NaN,NaN,NaN,총학 인스타그램 알려줘,https://www.instagram.com/jbnu_ch/,NaN
4,SNS안내,총학생회,NaN,NaN,NaN,총학 sns가 뭔지 알려줘,https://www.instagram.com/jbnu_ch/,NaN
...,...,...,...,...,...,...,...,...
1615,SNS안내,전북대학교,NaN,NaN,NaN,전북대 인스타 구해요,https://www.instagram.com/jbnu1947/,NaN
1616,SNS안내,전북대학교,NaN,NaN,NaN,전북대 인스타그램 구해요,https://www.instagram.com/jbnu1947/,NaN
1617,SNS안내,전북대학교,NaN,NaN,NaN,전북대 페북 구해요,https://www.instagram.com/jbnu1947/,NaN
1618,SNS안내,전북대학교,NaN,NaN,NaN,전북대 페이스북 구해요,https://www.instagram.com/jbnu1947/,NaN


In [7]:
#rawdata 통합
_df = pd.concat((train1,train2,train3,train4,train5,train6))[["의도","개체","시점","세부위치","답변","답변이미지"]]
#_df = pd.concat((train1,train2,train3,train4,train5))[["의도","개체","시점","세부위치","답변","답변이미지"]]
_df = _df.drop_duplicates(["의도","개체","시점","세부위치"])
train_df = _df
train_df
train_df.to_excel(train_file)

In [8]:
_df

,의도,개체,시점,세부위치,답변,답변이미지
0,교내학식,참빛관,전체,NaN,NaN,NaN
37,교내학식,참빛관,아침,NaN,NaN,NaN
84,교내학식,참빛관,점심,NaN,NaN,NaN
154,교내학식,참빛관,저녁,NaN,NaN,NaN
294,교내학식,한빛관,전체,NaN,NaN,NaN
...,...,...,...,...,...,...
2409,학과사무실,스마트팜학과,NaN,NaN,스마트팜학과 사무실\n사무실 : 농생명과학대학 1호관 402호 \nT 063-270...,NaN
0,교내식당 운영시간,생활관 참빛관 운영시간,NaN,NaN,"생활관 참빛관 운영시간\n조식 07:30~9:00, 중식 12:00~13:30, 석...",NaN
204,교내식당 운영시간,진수원 운영시간,NaN,NaN,"진수원 운영시간\n중식 11:30~14:00, 석식 17:30~19:00",NaN
510,교내식당 운영시간,생활관 직영관 운영시간,NaN,NaN,"생활관 직영관 운영시간\n조식 07:30~9:00, 중식 12:00~13:30, 석...",NaN


In [9]:
_df.의도.value_counts()

학과사무실        110
위치            78
교내학식          35
SNS안내          5
교내식당 운영시간      4
교내 공지          1
Name: 의도, dtype: int64

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 714
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   의도      233 non-null    object
 1   개체      232 non-null    object
 2   시점      35 non-null     object
 3   세부위치    42 non-null     object
 4   답변      197 non-null    object
 5   답변이미지   0 non-null      object
dtypes: object(6)
memory usage: 12.7+ KB


# 교내학식 답변 만들기
- 일단 임시로 답변 만들어 놓은 뒤 나중에 FindAnswer구문에서 시간을 체크하여 수정하고 가져오는 코드 만들기

# 참빛관

In [11]:
def get_chambit_menu():
    url1 = requests.get("https://likehome.jbnu.ac.kr/home/main/inner.php?sMenu=B7200")
    soup1 = bs(url1.text, "html.parser")
    data1 = soup1.find("table", {"class":"calendar_box"})

    table1 = pf.make2d(data1)
    df1 = pd.DataFrame(table1[1:], columns=table1[0])
    df1.columns =["시점"] +[str(chr) + "요일" for chr in ["일","월","화","수","목","금","토"]]
    df1 = df1.melt("시점")
    df1.columns = ["시점","요일","답변"]
    df1["의도"] = "교내학식"
    df1["개체"] = "참빛관" #바꿔야하는 위치
    df1["세부위치"] = pd.NA
    df1["답변이미지"] = pd.NA
    df1 = df1[["의도","개체","시점","요일","세부위치","답변","답변이미지"]]
    new_answer = []
    for idx,obs in df1.iterrows():
        txt = obs["개체"] + " " + obs["요일"] + " " +obs["시점"] +"\n" 
        menu = re.sub(" ","\n",obs["답변"])
        answer = txt + menu
        new_answer.append(answer)
    df1["답변"] = new_answer
    return df1

df = get_chambit_menu()
df

,의도,개체,시점,요일,세부위치,답변,답변이미지
0,교내학식,참빛관,아침,일요일,<NA>,참빛관 일요일 아침\n쌀밥\n떡만두국\n모듬야채햄볶음\n건강야채무침\n김치,<NA>
1,교내학식,참빛관,점심,일요일,<NA>,참빛관 일요일 점심\n고추참치볶음밥\n유부장국\n만두탕수\n양배추샐러드&포도드레싱\...,<NA>
2,교내학식,참빛관,저녁,일요일,<NA>,참빛관 일요일 저녁\n쌀밥\n두부감자된장국\n맥적구이&데리야끼소스\n치커리무침\n열...,<NA>
3,교내학식,참빛관,아침,월요일,<NA>,참빛관 월요일 아침\n쌀밥\n곰국\n양념팝콘치킨\n유부냉채\n오복지무침\n깍두기\n...,<NA>
4,교내학식,참빛관,점심,월요일,<NA>,참빛관 월요일 점심\n쌀밥\n돼지국밥\n사각어묵볶음\n부추무침\n배추나물\n깍두기\...,<NA>
5,교내학식,참빛관,저녁,월요일,<NA>,참빛관 월요일 저녁\n기장밥\n닭개장\n옥수수빠스\n모듬버섯볶음\n다시마채무침\n깍...,<NA>
6,교내학식,참빛관,아침,화요일,<NA>,참빛관 화요일 아침\n쌀밥\n시금치된장국\n어묵전\n맛살감자채볶음\n매운콩나물무침\...,<NA>
7,교내학식,참빛관,점심,화요일,<NA>,참빛관 화요일 점심\n쌀밥\n어묵국\n모듬장조림\n쥐어채볶음\n다시마튀각\n김치\n...,<NA>
8,교내학식,참빛관,저녁,화요일,<NA>,참빛관 화요일 저녁\n쌀밥\n배추된장국\n계란장조림\n깐마늘절임\n도토리묵&양념장\...,<NA>
9,교내학식,참빛관,아침,수요일,<NA>,참빛관 수요일 아침\n쌀밥\n떡국\n동그랑땡야채조림\n상추무침\n무말랭이무침\n김치...,<NA>


# 직영관
- 주말은 밥안나옴

In [12]:
#직영관 메뉴 가져오기
def get_jikyoung_menu():
    url1 = requests.get("https://likehome.jbnu.ac.kr/home/main/inner.php?sMenu=B7100")
    soup1 = bs(url1.text, "html.parser")
    data1 = soup1.find("table", {"class":"calendar_box"})

    table1 = pf.make2d(data1)
    df1 = pd.DataFrame(table1[1:], columns=table1[0])
    df1.columns =["시점"] +[str(chr) + "요일" for chr in ["일","월","화","수","목","금","토"]]
    df1 = df1.melt("시점")
    df1.columns = ["시점","요일","답변"]
    df1["의도"] = "교내학식"
    df1["개체"] = "직영관" #바꿔야하는 위치
    df1["세부위치"] = pd.NA
    df1["답변이미지"] = pd.NA
    df1 = df1[["의도","개체","시점","요일","세부위치","답변","답변이미지"]]
    new_answer = []
    for idx,obs in df1.iterrows():
        if obs["요일"] == "일요일" or obs["요일"] == "토요일":
            txt = "직영관은 " + obs["요일"] + "은 운영하지 않습니다."
            new_answer.append(txt)
        else:
            txt = obs["개체"] + " " + obs["요일"] + " " +obs["시점"] +"\n" 
            menu = re.sub(" ","\n",obs["답변"])
            answer = txt + menu
            new_answer.append(answer)
    df1["답변"] = new_answer
    return df1
_df = get_jikyoung_menu()
_df

,의도,개체,시점,요일,세부위치,답변,답변이미지
0,교내학식,직영관,아침,일요일,<NA>,직영관은 일요일은 운영하지 않습니다.,<NA>
1,교내학식,직영관,점심,일요일,<NA>,직영관은 일요일은 운영하지 않습니다.,<NA>
2,교내학식,직영관,저녁,일요일,<NA>,직영관은 일요일은 운영하지 않습니다.,<NA>
3,교내학식,직영관,아침,월요일,<NA>,직영관 월요일 아침\n흰밥\n북어해장국\n오징어떡볶음\n스팸계란전\n포기김치\n우유...,<NA>
4,교내학식,직영관,점심,월요일,<NA>,직영관 월요일 점심\n미역장국\n불닭볶음밥*눈꽃치즈\n피쉬앤칩스\n치커리사과유자무침...,<NA>
5,교내학식,직영관,저녁,월요일,<NA>,직영관 월요일 저녁\n흰밥\n얼큰짬뽕국\n훈제오리야채볶음\n달걀토마토스크램블\n포기...,<NA>
6,교내학식,직영관,아침,화요일,<NA>,직영관 화요일 아침\n잡곡밥\n참치감자찌개\n간장소불고기\n콩나물무침\n포기김치\n...,<NA>
7,교내학식,직영관,점심,화요일,<NA>,직영관 화요일 점심\n잡곡밥\n건새우아욱국\n돈수육*야채쌈\n한식잡채\n무말랭이무침...,<NA>
8,교내학식,직영관,저녁,화요일,<NA>,직영관 화요일 저녁\n콩나물국\n새우볶음밥\n사천탕수육\n열대과일그린샐러드\n포기김...,<NA>
9,교내학식,직영관,아침,수요일,<NA>,직영관 수요일 아침\n흰밥\n순두부찌개\n통통고기완자전*케찹\n베이컨감자볶음\n포기...,<NA>


# 진수원 

In [13]:
#진수원 메뉴 가져오기
def get_jinsuwon_menu():
    #크롤링
    url3 = urlopen("https://sobi.chonbuk.ac.kr/menu/week_menu.php")
    soup3 = bs(url3.read(), 'html.parser')
    data3 = soup3.find("table", {"class":"tblType03"})
    table3 = pf.make2d(data3)
    df3 = pd.DataFrame(table3[1:], columns=table3[0])
    #이름통일
    day_mapper={}
    for chr in ["월","화","수","목","금","토","일"]:
        day_mapper[chr] = chr+"요일"
    #컬럼통일
    df3 = df3.rename(columns=day_mapper)
    df3 = df3.iloc[:,[0,2,3,4,5,6]]
    df3.loc[:,"식구분"] = ["점심","저녁"]
    df3 = df3.melt("식구분").rename(columns = {"식구분":"시점","variable":"요일","value":"답변"})
    df3["의도"] = "교내학식"
    df3["개체"] = "진수원" #바꿔야하는 위치
    df3["세부위치"] = pd.NA
    df3["답변이미지"] = pd.NA
    df3 = df3[["의도","개체","시점","요일","세부위치","답변","답변이미지"]]
    new_answer = []
    for idx,obs in df3.iterrows():
        txt = obs["개체"] + " " + obs["요일"] + " " +obs["시점"] +"\n"
        menu = obs["답변"]
        answer = txt + menu
        new_answer.append(answer)
    df3["답변"] = new_answer
    _sat_sun = pd.DataFrame(data=[["교내학식","진수원","전체","토요일",pd.NA,"진수원은 주말에는 운영하지 않습니다.",pd.NA],\
        ["교내학식","진수원","전체","일요일",pd.NA,"진수원은 주말에는 운영하지 않습니다.",pd.NA]],columns = ["의도","개체","시점","요일","세부위치","답변","답변이미지"])
    _df = pd.concat([df3,_sat_sun],axis=0).reset_index(drop=True)
    return _df

_df = get_jinsuwon_menu()
_df

,의도,개체,시점,요일,세부위치,답변,답변이미지
0,교내학식,진수원,점심,월요일,<NA>,진수원 월요일 점심\n옹심이떡국치즈너비아니조림통영식오징어무침참나물무침,<NA>
1,교내학식,진수원,저녁,월요일,<NA>,진수원 월요일 저녁\n새우볶음밥*미역두부장국불닭순대볶음햄감자채볶음실파김무침,<NA>
2,교내학식,진수원,점심,화요일,<NA>,진수원 화요일 점심\n동태찌개돈사태단호박조림새우까스샐러드느타리숙주무침,<NA>
3,교내학식,진수원,저녁,화요일,<NA>,진수원 화요일 저녁\n잔치국수돈육김치볶음*찐두부만두탕수육오복채무침,<NA>
4,교내학식,진수원,점심,수요일,<NA>,진수원 수요일 점심\n온메밀소바불맛돈불고기계란찜청경채나물,<NA>
5,교내학식,진수원,저녁,수요일,<NA>,진수원 수요일 저녁\n참치김치찌개*라면사리돈까스&샐러드아비꼬카레소시지떡볶음,<NA>
6,교내학식,진수원,점심,목요일,<NA>,진수원 목요일 점심\n아욱된장국꽃도리탕메이플크로켓시금치단감샐러드,<NA>
7,교내학식,진수원,저녁,목요일,<NA>,진수원 목요일 저녁\n돈등뼈감자탕뿌링클치킨불닭팽이구이단호박샐러드,<NA>
8,교내학식,진수원,점심,금요일,<NA>,진수원 금요일 점심\n바지락미역국쇠고기버섯볶음마파두부흑임자드레싱파스타샐러드,<NA>
9,교내학식,진수원,저녁,금요일,<NA>,진수원 금요일 저녁\n전복영양죽묵은지닭조림꽈리어묵볶음오이소박이,<NA>


In [14]:
"""
#의대 메뉴 가져오기 => 안됨
def get_jinsuwon_menu():
    #크롤링
    url3 = urlopen("https://sobi.chonbuk.ac.kr/menu/week_menu.php")
    soup3 = bs(url3.read(), 'html.parser')
    data3 = soup3.find("table", {"class":"tblType03"})
    table3 = pf.make2d(data3)
    df3 = pd.DataFrame(table3[1:], columns=table3[0])
    #이름통일
    day_mapper={}
    for chr in ["월","화","수","목","금","토","일"]:
        day_mapper[chr] = chr+"요일"
    #컬럼통일
    df3 = df3.rename(columns=day_mapper)
    df3 = df3.iloc[:,[0,2,3,4,5,6]]
    df3.loc[:,"식구분"] = ["점심","저녁"]
    df3 = df3.melt("식구분").rename(columns = {"식구분":"시점","variable":"요일","value":"답변"})
    df3["의도"] = "교내학식"
    df3["개체"] = "의대" #바꿔야하는 위치
    df3["세부위치"] = pd.NA
    df3["답변이미지"] = pd.NA
    df3 = df3[["의도","개체","시점","요일","세부위치","답변","답변이미지"]]
    return df3

_df = get_jinsuwon_menu()
_df
"""

'\n#의대 메뉴 가져오기 => 안됨\ndef get_jinsuwon_menu():\n    #크롤링\n    url3 = urlopen("https://sobi.chonbuk.ac.kr/menu/week_menu.php")\n    soup3 = bs(url3.read(), \'html.parser\')\n    data3 = soup3.find("table", {"class":"tblType03"})\n    table3 = pf.make2d(data3)\n    df3 = pd.DataFrame(table3[1:], columns=table3[0])\n    #이름통일\n    day_mapper={}\n    for chr in ["월","화","수","목","금","토","일"]:\n        day_mapper[chr] = chr+"요일"\n    #컬럼통일\n    df3 = df3.rename(columns=day_mapper)\n    df3 = df3.iloc[:,[0,2,3,4,5,6]]\n    df3.loc[:,"식구분"] = ["점심","저녁"]\n    df3 = df3.melt("식구분").rename(columns = {"식구분":"시점","variable":"요일","value":"답변"})\n    df3["의도"] = "교내학식"\n    df3["개체"] = "의대" #바꿔야하는 위치\n    df3["세부위치"] = pd.NA\n    df3["답변이미지"] = pd.NA\n    df3 = df3[["의도","개체","시점","요일","세부위치","답변","답변이미지"]]\n    return df3\n\n_df = get_jinsuwon_menu()\n_df\n'

# 후생관 메뉴 가져오기

In [15]:
def get_hoosangwan_menu():
    #후생관 데이터 불러오기
    url3 = urlopen("https://sobi.chonbuk.ac.kr/menu/week_menu.php")
    soup3 = bs(url3.read(), 'html.parser')
    data5 = soup3.find("div", {"class":"table_wrap"})
    table5 = pf.make2d(data5)
    df5 = pd.DataFrame(table5[1:], columns=table5[0])
    #print(df5)
    df5.columns = ["식구분1","식구분"] + [chr + "요일" for chr in ["월","화","수","목","금"]]
    df5 = df5.iloc[:,1:].dropna(axis='index',thresh=1).melt(("식구분"))
    df5.columns = ["식구분","요일","답변"]
    days = df5.요일.unique()

    fixed_menu = {}
    data = []
    for _,obs in df5.iterrows():
        #매일 같은 분식,양식,라면의 고정메뉴가 월요일에만 있음. 따라서 월요일에서 고정메뉴를 가져옴
        if (obs["식구분"] == "분식" or obs["식구분"] == "양식" or obs["식구분"] == "라면") and obs["요일"] == "월요일": 
            fixed_menu[obs["식구분"]] = obs["답변"]
    #같이 처리할수도 있을 것 같지만 ㅜㅜ 따로처리
    for _,obs in df5.iterrows():
        day_menu = ""
        timepoint = "전체";day = obs["요일"];answer = obs["답변"];setting = obs["식구분"]
        dt = [setting,timepoint,day,answer]
        data.append(dt)
    _df = pd.DataFrame(data)
    _df.columns = ["식구분","시점","요일","답변"]
    days = _df.요일.unique()
    answers = []
    for day in days:
        cond_raw = (_df["요일"] == day) & ~((_df["식구분"] == "분식") | (_df["식구분"] == "양식") | (_df["식구분"] == "라면")) #분식,양식,라면은 고정메뉴이므로 pass
        day_df = _df[cond_raw]
        answer = str(day)+" 후생관 메뉴\n"
        for _,obs in day_df.iterrows():
            식구분 = obs["식구분"]
            유동메뉴 = obs["답변"]
            answer += 식구분+"\n"+유동메뉴 +"\n"
            
        fixed_txt = "고정메뉴\n"
            
        for _tp,_menu in fixed_menu.items():
            fixed_txt += _tp + "\n" + _menu + "\n"
            
        answer = answer + fixed_txt
                #print(_tp,_menu)
        answers.append(answer)
    _df = pd.DataFrame({"요일" : days,"답변" : answers})
    _df["의도"] = "교내학식"
    _df["개체"] = "후생관" #바꿔야하는 위치
    _df["세부위치"] = pd.NA
    _df["답변이미지"] = pd.NA
    _df["시점"] = "전체"
    _df = _df[["의도","개체","시점","요일","세부위치","답변","답변이미지"]]
    _sat_sun = pd.DataFrame(data=[["교내학식","후생관","전체","토요일",pd.NA,"후생관은 주말에는 운영하지 않습니다.",pd.NA],\
        ["교내학식","후생관","전체","일요일",pd.NA,"후생관은 주말에는 운영하지 않습니다.",pd.NA]],columns = ["의도","개체","시점","요일","세부위치","답변","답변이미지"])
    _df = pd.concat((_df,_sat_sun),axis=0)
    _df = _df.reset_index(drop = True)
    return _df

_df = get_hoosangwan_menu()
_df

,의도,개체,시점,요일,세부위치,답변,답변이미지
0,교내학식,후생관,전체,월요일,<NA>,월요일 후생관 메뉴\n찌개\n등뼈감자탕&라면사리\n돌솥\n운영없음\n특 식\n삼겹살...,<NA>
1,교내학식,후생관,전체,화요일,<NA>,화요일 후생관 메뉴\n찌개\n운영없음\n돌솥\n불닭마요돌솥\n특 식\n바싹불고기김치...,<NA>
2,교내학식,후생관,전체,수요일,<NA>,수요일 후생관 메뉴\n찌개\n삼겹살김치찌개&계란후라이\n돌솥\n운영없음\n특 식\n...,<NA>
3,교내학식,후생관,전체,목요일,<NA>,목요일 후생관 메뉴\n찌개\n사골순대국밥&요구르트\n돌솥\n김치베이컨치즈돌솥\n특 ...,<NA>
4,교내학식,후생관,전체,금요일,<NA>,금요일 후생관 메뉴\n찌개\n운영없음\n돌솥\n뚝배기치즈함박\n특 식\n치킨가라아게...,<NA>
5,교내학식,후생관,전체,토요일,<NA>,후생관은 주말에는 운영하지 않습니다.,<NA>
6,교내학식,후생관,전체,일요일,<NA>,후생관은 주말에는 운영하지 않습니다.,<NA>


## 소문자로 변환해주는 함수.

In [16]:
def to_lower():
    train_data = pd.read_excel(train_file).drop(columns = ["Unnamed: 0"])
    #print(train_data)
    for var_name,var_values in train_data.iteritems():
        lower_values = []
        if (var_name != "답변") and (var_name != "답변이미지"):
            for value in var_values:
                try:
                    if value.isupper():
                        #print(value)
                        lower_values.append(value.lower())
                    else:
                        lower_values.append(value)
                except: #NA값이 뜰 경우 오류가 나므로 예외처리//그대로 추가
                    lower_values.append(value)
            #print(len(train_data),len(lower_values),var_name)
            train_data[var_name] = lower_values
    train_data.to_excel(train_file)
    return train_data

## 메뉴업데이트 함수

In [17]:
def menu_update(): 
    """
    챗봇학습용(답변용)파일에서 intent == "교내학식"인 데이터들의 메뉴 업데이트
    단,xlsx파일 업데이트임. Mysql DB에는 따로 적용해줘야 함
    """
    #0.기존 답변용 데이터 불러오고
    train_data = pd.read_excel(train_file)
    train_data = train_data.drop(columns = "Unnamed: 0")
    #1.데이터에서 intent == "교내학식"인 row를 모두 삭제
    _idx = train_data[train_data.의도 == "교내학식"].index
    train_data = train_data.drop(index = _idx)

    #2.각 식당별 메뉴 가져오기
    chambit_menu = get_chambit_menu()
    jikyoun_menu = get_jikyoung_menu()
    jinsuwon_menu = get_jinsuwon_menu()
    hoosangwan_menu = get_hoosangwan_menu()

    #3.합쳐서 챗봇학습(답변용)데이터 만들기
    menu_train_data = pd.concat([chambit_menu,jikyoun_menu,jinsuwon_menu,hoosangwan_menu]).reset_index(drop=True)
    updated_train_data = pd.concat((train_data,menu_train_data),axis=0)[["의도","개체","시점","요일","세부위치","답변","답변이미지"]]
    updated_train_data = updated_train_data.dropna(subset = ["의도"]).reset_index(drop=True)

    #4보류
    """
    #4.의도 == "교내학식" , 시점 == "아침 or 점심 or 저녁 " ,요일 == "해당하는요일"인 데이터셋 만들기
    #발화의 의도가 교내학식일 경우만을 위함
    cond = updated_train_data["의도"] == "교내학식"
    menu_data = updated_train_data[cond]

    days = menu_data["요일"].unique()
    timepoints = menu_data["시점"].unique()
    print(timepoints)
    for day in days:
        for tp in timepoints:
            #answer = day + " " + tp + " 학식\n
            answer = ""
            #print(answer)
            cond = (menu_data["요일"] == day) & (menu_data["시점"] == tp)
            target_df = menu_data[cond]
            for _,obs in target_df.iterrows():
                answer += obs["답변"] +"\n"
                print(obs)
                #print(obs)
    """
    #5.모두 소문자로 변환
    
    print("의도 널 갯수:",sum(pd.isna(_df.의도)))
    print("답변 널 갯수:",sum(pd.isna(_df.답변)))
    updated_train_data.to_excel(train_file)
    train_data = to_lower()
    
    
    return train_data

updated_menu = menu_update()
updated_menu

의도 널 갯수: 0
답변 널 갯수: 0


,의도,개체,시점,요일,세부위치,답변,답변이미지
0,교내 공지,NaN,NaN,NaN,NaN,NaN,NaN
1,위치,전북대학교 간호대학,NaN,NaN,NaN,http://kko.to/roHTsJwxoC,NaN
2,위치,전북대학교 사범대학,NaN,NaN,NaN,http://kko.to/bO0ulzz657sm,NaN
3,위치,전북대학교 인문대학,NaN,NaN,1호관,http://kko.to/SpLnzD3vLw,NaN
4,위치,전북대학교 인문대학,NaN,NaN,2호관,http://kko.to/yhJvosYoxz,NaN
...,...,...,...,...,...,...,...
254,교내학식,후생관,전체,수요일,NaN,수요일 후생관 메뉴\n찌개\n삼겹살김치찌개&계란후라이\n돌솥\n운영없음\n특 식\n...,NaN
255,교내학식,후생관,전체,목요일,NaN,목요일 후생관 메뉴\n찌개\n사골순대국밥&요구르트\n돌솥\n김치베이컨치즈돌솥\n특 ...,NaN
256,교내학식,후생관,전체,금요일,NaN,금요일 후생관 메뉴\n찌개\n운영없음\n돌솥\n뚝배기치즈함박\n특 식\n치킨가라아게...,NaN
257,교내학식,후생관,전체,토요일,NaN,후생관은 주말에는 운영하지 않습니다.,NaN


In [18]:
cond = updated_menu["의도"] == "교내학식"
updated_menu[cond]

,의도,개체,시점,요일,세부위치,답변,답변이미지
198,교내학식,참빛관,아침,일요일,NaN,참빛관 일요일 아침\n쌀밥\n떡만두국\n모듬야채햄볶음\n건강야채무침\n김치,NaN
199,교내학식,참빛관,점심,일요일,NaN,참빛관 일요일 점심\n고추참치볶음밥\n유부장국\n만두탕수\n양배추샐러드&포도드레싱\...,NaN
200,교내학식,참빛관,저녁,일요일,NaN,참빛관 일요일 저녁\n쌀밥\n두부감자된장국\n맥적구이&데리야끼소스\n치커리무침\n열...,NaN
201,교내학식,참빛관,아침,월요일,NaN,참빛관 월요일 아침\n쌀밥\n곰국\n양념팝콘치킨\n유부냉채\n오복지무침\n깍두기\n...,NaN
202,교내학식,참빛관,점심,월요일,NaN,참빛관 월요일 점심\n쌀밥\n돼지국밥\n사각어묵볶음\n부추무침\n배추나물\n깍두기\...,NaN
...,...,...,...,...,...,...,...
254,교내학식,후생관,전체,수요일,NaN,수요일 후생관 메뉴\n찌개\n삼겹살김치찌개&계란후라이\n돌솥\n운영없음\n특 식\n...,NaN
255,교내학식,후생관,전체,목요일,NaN,목요일 후생관 메뉴\n찌개\n사골순대국밥&요구르트\n돌솥\n김치베이컨치즈돌솥\n특 ...,NaN
256,교내학식,후생관,전체,금요일,NaN,금요일 후생관 메뉴\n찌개\n운영없음\n돌솥\n뚝배기치즈함박\n특 식\n치킨가라아게...,NaN
257,교내학식,후생관,전체,토요일,NaN,후생관은 주말에는 운영하지 않습니다.,NaN


In [19]:
updated_menu["의도"].value_counts()

학과사무실        110
위치            78
교내학식          61
sns안내          5
교내식당 운영시간      4
교내 공지          1
Name: 의도, dtype: int64

## 교내공지 업데이트

In [20]:
def notice_update():
    #먼저 데이터 불러오기
    train_data = pd.read_excel(train_file)
    train_data = train_data.drop(columns = "Unnamed: 0")
    #크롤링
    response = requests.get("https://www.jbnu.ac.kr/kor/?menuID=139")
    html = response.text
    soup = bs(html,'html5lib')
    result = soup.select("div table tbody td span a",href = True)
    #print(result[0]['href'])
    #print(type(result[0]))

    notice_answer = ""
    for idx,r in enumerate(result):
        #print(f"{idx} ==================")
        #print(r.text)
        jb_url = "https://www.jbnu.ac.kr/kor/"
        last_url = r["href"]
        url = jb_url +last_url

        txt = r.text
        txt = re.sub("\\n","",txt)
        txt = re.sub("\\t","",txt)
        notice_answer += txt + "\n"
        notice_answer += url + "\n"
        #print(txt)
        #print(url)
    #print(notice_answer)
    train_data.loc[train_data["의도"] == "교내 공지","답변"] = notice_answer
    train_data.to_excel(train_file)
        
notice_update()